In [2]:
import os 
import numpy as np
import pandas as pd

In [4]:
lt_logs_df = pd.read_csv('./logs/long_term_forecast.csv', index_col = 0)
from types import SimpleNamespace
args = SimpleNamespace(**lt_logs_df.loc[21].to_dict())

In [ ]:
sequence_len = 20
target_len = 20 

args.seq_len = sequence_len
args.pred_len = target_len
root_path = './dataset/USC/'
